In [1]:
import pandas as pd
# Общие библиотеки
import re
import os
import cv2
import numpy as np

# Для работы с pdf
from pdf2image import convert_from_path


# Для работы с изображениями
import pytesseract
from PIL import Image, ImageDraw, ImageFilter, ImageEnhance

# Для работы с почтой
import imaplib
import email
from email.header import decode_header

import locale

In [2]:
import cv2
import pytesseract
import os

def preprocess_image(image_path, save_path=None):
    # Загрузка изображения
    img = cv2.imread(image_path)
    
    # Преобразование в градации серого
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Масштабирование
    gray = cv2.resize(gray, None, fx=1.5, fy=1.5, interpolation=cv2.INTER_LINEAR)

    # Адаптивная пороговая обработка
    thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 31, 2)

    # Морфологические операции
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, 1))
    thresh = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)

    # Устранение шума
    thresh = cv2.medianBlur(thresh, 3)

    # Сохранение обработанного изображения, если указан путь
    if save_path is not None:
        cv2.imwrite(save_path, thresh)

    return thresh

def extract_text_from_image(preprocessed_img):
    config = r'--oem 3 --psm 6'
    data = pytesseract.image_to_data(preprocessed_img, config=config, lang='rus')

    text_data = []
    last_y = 0
    for i, el in enumerate(data.splitlines()):
        if i == 0:
            continue

        el = el.split()
        if len(el) == 12:
            x, y, w, h, text = int(el[6]), int(el[7]), int(el[8]), int(el[9]), el[11]
            if y > last_y + h:  # Новая строка
                text_data.append('\n')
            last_y = y
            text_data.append(text)

    return ' '.join(text_data)

def save_text_to_file(text, file_path):
    with open(file_path, 'w', encoding='utf-8') as file:
        file.write(text)

def process_images_from_folder(input_folder, output_folder, save_processed_folder):
    all_files = os.listdir(input_folder)
    jpg_files = [f for f in all_files if f.endswith('.jpg')]

    for file in jpg_files:
        file_path = os.path.join(input_folder, file)
        save_processed_path = os.path.join(save_processed_folder, file)
        
        # Предобработка и сохранение изображения
        preprocessed_img = preprocess_image(file_path, save_processed_path)
        extracted_text = extract_text_from_image(preprocessed_img)  # Здесь нужно обновить функцию extract_text_from_image

        # Создаем текстовый файл с тем же именем, что и изображение
        text_file_name = os.path.splitext(file)[0] + ".txt"
        text_file_path = os.path.join(output_folder, text_file_name)

        save_text_to_file(extracted_text, text_file_path)
        print(f"Текст из {file} сохранен в {text_file_path}")

# Указываем пути к папкам
input_folder = r'C:\Users\NeKonn\orders\PDF_parser\Output_Images'
output_folder = r'C:\Users\NeKonn\orders\PDF_parser\test\TEXT'
save_processed_folder = r'C:\Users\NeKonn\orders\PDF_parser\Processed_Images'

# Создаем папки для сохранения обработанных изображений, если они еще не существуют
if not os.path.exists(save_processed_folder):
    os.makedirs(save_processed_folder)

process_images_from_folder(input_folder, output_folder, save_processed_folder)

Текст из 1.jpg сохранен в C:\Users\NeKonn\orders\PDF_parser\test\TEXT\1.txt
Текст из 10.jpg сохранен в C:\Users\NeKonn\orders\PDF_parser\test\TEXT\10.txt
Текст из 11.jpg сохранен в C:\Users\NeKonn\orders\PDF_parser\test\TEXT\11.txt
Текст из 12.jpg сохранен в C:\Users\NeKonn\orders\PDF_parser\test\TEXT\12.txt
Текст из 2.jpg сохранен в C:\Users\NeKonn\orders\PDF_parser\test\TEXT\2.txt
Текст из 3.jpg сохранен в C:\Users\NeKonn\orders\PDF_parser\test\TEXT\3.txt
Текст из 4.jpg сохранен в C:\Users\NeKonn\orders\PDF_parser\test\TEXT\4.txt
Текст из 5.jpg сохранен в C:\Users\NeKonn\orders\PDF_parser\test\TEXT\5.txt
Текст из 6.jpg сохранен в C:\Users\NeKonn\orders\PDF_parser\test\TEXT\6.txt
Текст из 7.jpg сохранен в C:\Users\NeKonn\orders\PDF_parser\test\TEXT\7.txt
Текст из 8.jpg сохранен в C:\Users\NeKonn\orders\PDF_parser\test\TEXT\8.txt
Текст из 9.jpg сохранен в C:\Users\NeKonn\orders\PDF_parser\test\TEXT\9.txt


In [3]:
def extract_text_from_images_with_word_boxes(folder_path, text_folder_path, boxes_folder_path):
#     pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

    if not os.path.exists(text_folder_path):
        os.makedirs(text_folder_path)
    if not os.path.exists(boxes_folder_path):
        os.makedirs(boxes_folder_path)

    for image_file in os.listdir(folder_path):
        if image_file.endswith('.jpg'):
            image_path = os.path.join(folder_path, image_file)
            image = Image.open(image_path)

            # Извлечение данных о боксах и тексте
            data = pytesseract.image_to_data(image, lang='rus')

            # Рисование боксов на изображении и сбор текста из боксов
            draw = ImageDraw.Draw(image)
            words_in_boxes = []  # Список для сохранения слов в боксах
            for i, item in enumerate(data.splitlines()):
                if i == 0:
                    continue
                item = item.split()
                if len(item) == 12 and item[11] != '-1':  # Проверка, что строка содержит данные о слове
                    x, y, w, h = int(item[6]), int(item[7]), int(item[8]), int(item[9])
                    draw.rectangle(((x, y), (x + w, y + h)), outline="red")
                    words_in_boxes.append(item[11])  # Добавление слова в список

            # Сохранение изображения с боксами
            boxes_image_path = os.path.join(boxes_folder_path, image_file)
            image.save(boxes_image_path)

            # Сохранение текста слов в боксах в файл
            text_file_name = os.path.splitext(image_file)[0] + '.txt'
            text_file_path = os.path.join(text_folder_path, text_file_name)
            with open(text_file_path, 'w', encoding='utf-8') as file:
                file.write(' '.join(words_in_boxes))  # Сохранение всех слов, найденных в боксах

            print(f"Текст из боксов {image_file} сохранен в {text_file_path}")
            print(f"Изображение с боксами сохранено в {boxes_image_path}")

# Пути к папкам
image_folder = r'C:\Users\NeKonn\orders\PDF_parser\Output_Images'
text_folder = r'C:\Users\NeKonn\orders\PDF_parser\TXT'
boxes_folder = r'C:\Users\NeKonn\orders\PDF_parser\Boxes_Images'

# Вызов функции
extract_text_from_images_with_word_boxes(image_folder, text_folder, boxes_folder)

Текст из боксов 1.jpg сохранен в C:\Users\NeKonn\orders\PDF_parser\TXT\1.txt
Изображение с боксами сохранено в C:\Users\NeKonn\orders\PDF_parser\Boxes_Images\1.jpg
Текст из боксов 10.jpg сохранен в C:\Users\NeKonn\orders\PDF_parser\TXT\10.txt
Изображение с боксами сохранено в C:\Users\NeKonn\orders\PDF_parser\Boxes_Images\10.jpg
Текст из боксов 11.jpg сохранен в C:\Users\NeKonn\orders\PDF_parser\TXT\11.txt
Изображение с боксами сохранено в C:\Users\NeKonn\orders\PDF_parser\Boxes_Images\11.jpg
Текст из боксов 12.jpg сохранен в C:\Users\NeKonn\orders\PDF_parser\TXT\12.txt
Изображение с боксами сохранено в C:\Users\NeKonn\orders\PDF_parser\Boxes_Images\12.jpg
Текст из боксов 2.jpg сохранен в C:\Users\NeKonn\orders\PDF_parser\TXT\2.txt
Изображение с боксами сохранено в C:\Users\NeKonn\orders\PDF_parser\Boxes_Images\2.jpg
Текст из боксов 3.jpg сохранен в C:\Users\NeKonn\orders\PDF_parser\TXT\3.txt
Изображение с боксами сохранено в C:\Users\NeKonn\orders\PDF_parser\Boxes_Images\3.jpg
Текст и

In [4]:
def remove_lines(image):
    # Конвертирование изображения в градации серого и бинаризация
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

    # Определение и удаление горизонтальных линий
    horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (25, 1))
    detected_lines = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, horizontal_kernel, iterations=2)
    cnts = cv2.findContours(detected_lines, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    for c in cnts:
        cv2.drawContours(image, [c], -1, (255, 255, 255), 5)

    # Определение и удаление вертикальных линий
    vertical_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, 25))
    detected_lines = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, vertical_kernel, iterations=2)
    cnts = cv2.findContours(detected_lines, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    for c in cnts:
        cv2.drawContours(image, [c], -1, (255, 255, 255), 5)

    return image

# Функция для извлечения текста и рисования боксов
def extract_text_and_draw_boxes(image_path, text_folder_path, boxes_folder_path):
    # Удаление линий
    image = cv2.imread(image_path)
    no_lines_image = remove_lines(image)

    # Конвертация обратно в формат PIL для использования с pytesseract
    image_pil = Image.fromarray(cv2.cvtColor(no_lines_image, cv2.COLOR_BGR2RGB))

    # Извлечение данных о боксах и тексте
    data = pytesseract.image_to_data(image_pil, lang='rus', output_type=pytesseract.Output.DICT)

    # Рисование боксов на изображении и сбор текста из боксов
    draw = ImageDraw.Draw(image_pil)
    words_in_boxes = []  # Список для сохранения слов в боксах

    for i in range(len(data['level'])):
        if data['text'][i] != '' and int(data['conf'][i]) > 0:  # Убедитесь, что конфиденциальность выше 0
            (x, y, w, h) = (data['left'][i], data['top'][i], data['width'][i], data['height'][i])
            draw.rectangle(((x, y), (x + w, y + h)), outline="red")
            words_in_boxes.append(data['text'][i])

    # Сохранение изображения с боксами
    boxes_image_path = os.path.join(boxes_folder_path, os.path.basename(image_path))
    image_pil.save(boxes_image_path)

    # Сохранение текста слов в боксах в файл
    text_file_name = os.path.splitext(os.path.basename(image_path))[0] + '.txt'
    text_file_path = os.path.join(text_folder_path, text_file_name)
    with open(text_file_path, 'w', encoding='utf-8') as file:
        file.write(' '.join(words_in_boxes))  # Сохранение всех слов, найденных в боксах

    print(f"Текст из боксов сохранен в {text_file_path}")
    print(f"Изображение с боксами сохранено в {boxes_image_path}")

# Пути к папкам
image_folder = r'C:\Users\NeKonn\orders\PDF_parser\Output_Images'
text_folder = r'C:\Users\NeKonn\orders\PDF_parser\TXT'
boxes_folder = r'C:\Users\NeKonn\orders\PDF_parser\Boxes_Images'

# Создание папок, если они еще не существуют
if not os.path.exists(text_folder):
    os.makedirs(text_folder)
if not os.path.exists(boxes_folder):
    os.makedirs(boxes_folder)

# Процесс обработки изображений
for image_file in os.listdir(image_folder):
    if image_file.endswith('.jpg'):
        image_path = os.path.join(image_folder, image_file)
        extract_text_and_draw_boxes(image_path, text_folder, boxes_folder)

Текст из боксов сохранен в C:\Users\NeKonn\orders\PDF_parser\TXT\1.txt
Изображение с боксами сохранено в C:\Users\NeKonn\orders\PDF_parser\Boxes_Images\1.jpg
Текст из боксов сохранен в C:\Users\NeKonn\orders\PDF_parser\TXT\10.txt
Изображение с боксами сохранено в C:\Users\NeKonn\orders\PDF_parser\Boxes_Images\10.jpg
Текст из боксов сохранен в C:\Users\NeKonn\orders\PDF_parser\TXT\11.txt
Изображение с боксами сохранено в C:\Users\NeKonn\orders\PDF_parser\Boxes_Images\11.jpg
Текст из боксов сохранен в C:\Users\NeKonn\orders\PDF_parser\TXT\12.txt
Изображение с боксами сохранено в C:\Users\NeKonn\orders\PDF_parser\Boxes_Images\12.jpg
Текст из боксов сохранен в C:\Users\NeKonn\orders\PDF_parser\TXT\2.txt
Изображение с боксами сохранено в C:\Users\NeKonn\orders\PDF_parser\Boxes_Images\2.jpg
Текст из боксов сохранен в C:\Users\NeKonn\orders\PDF_parser\TXT\3.txt
Изображение с боксами сохранено в C:\Users\NeKonn\orders\PDF_parser\Boxes_Images\3.jpg
Текст из боксов сохранен в C:\Users\NeKonn\ord

In [5]:
# def remove_lines(image):
#     # Конвертирование изображения в градации серого и бинаризация
#     gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
#     thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

#     # Определение и удаление горизонтальных линий
#     horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (25, 1))
#     detected_lines = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, horizontal_kernel, iterations=2)
#     cnts = cv2.findContours(detected_lines, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
#     cnts = cnts[0] if len(cnts) == 2 else cnts[1]
#     for c in cnts:
#         cv2.drawContours(image, [c], -1, (255, 255, 255), 5)

#     # Определение и удаление вертикальных линий
#     vertical_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, 25))
#     detected_lines = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, vertical_kernel, iterations=2)
#     cnts = cv2.findContours(detected_lines, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
#     cnts = cnts[0] if len(cnts) == 2 else cnts[1]
#     for c in cnts:
#         cv2.drawContours(image, [c], -1, (255, 255, 255), 5)

#     return image

# # Функция для извлечения текста и рисования боксов
# def extract_text_and_draw_boxes(image_path, text_folder_path, boxes_folder_path):
#     # Удаление линий
#     image = cv2.imread(image_path)
#     no_lines_image = remove_lines(image)

#     # Конвертация обратно в формат PIL для использования с pytesseract
#     image_pil = Image.fromarray(cv2.cvtColor(no_lines_image, cv2.COLOR_BGR2RGB))

#     # Извлечение данных о боксах и тексте
#     data = pytesseract.image_to_data(image_pil, lang='rus', output_type=pytesseract.Output.DICT)

#     # Рисование боксов на изображении и сохранение текста по строкам
#     draw = ImageDraw.Draw(image_pil)
#     text_data = {}  # Словарь для группировки слов по Y-координате

#     for i in range(len(data['level'])):
#         if data['text'][i].strip() != '' and int(data['conf'][i]) > 0:
#             x, y, w, h, text = data['left'][i], data['top'][i], data['width'][i], data['height'][i], data['text'][i]
#             draw.rectangle(((x, y), (x + w, y + h)), outline="red")

#             # Группировка слов по строкам на основе Y-координаты
#             if y in text_data:
#                 text_data[y].append(text)
#             else:
#                 text_data[y] = [text]

#     # Сортировка словаря по ключам (Y-координатам) и объединение слов в строки
#     sorted_text = [text_data[key] for key in sorted(text_data.keys())]
#     text_lines = [' '.join(line) for line in sorted_text]

#     # Сохранение изображения с боксами
#     boxes_image_path = os.path.join(boxes_folder_path, os.path.basename(image_path))
#     image_pil.save(boxes_image_path)

#     # Сохранение текста слов в файл, с сохранением расположения строк
#     text_file_name = os.path.splitext(os.path.basename(image_path))[0] + '.txt'
#     text_file_path = os.path.join(text_folder_path, text_file_name)
#     with open(text_file_path, 'w', encoding='utf-8') as file:
#         file.write('\n'.join(text_lines))  # Сохранение текста с учетом строк

#     print(f"Текст из боксов сохранен в {text_file_path}")
#     print(f"Изображение с боксами сохранено в {boxes_image_path}")

# # Пути к папкам
# image_folder = r'C:\Users\NeKonn\orders\PDF_parser\Output_Images'
# text_folder = r'C:\Users\NeKonn\orders\PDF_parser\TXT'
# boxes_folder = r'C:\Users\NeKonn\orders\PDF_parser\Boxes_Images'

# # Создание папок, если они еще не существуют
# if not os.path.exists(text_folder):
#     os.makedirs(text_folder)
# if not os.path.exists(boxes_folder):
#     os.makedirs(boxes_folder)

# # Процесс обработки изображений
# for image_file in os.listdir(image_folder):
#     if image_file.endswith('.jpg'):
#         image_path = os.path.join(image_folder, image_file)
#         extract_text_and_draw_boxes(image_path, text_folder, boxes_folder)